# Extract Specific Analysis Results

This notebook imports the structure saved as pickle file. 
(It has to run in the 'strucenglib3' environment. As it needs compas fea and the strucEngLib to read the pickled object.) it also runs in the sampling environment... warum weis sich nicht

This script reads out specified values form the result dict of each structure of a samiling iteration and saves the results in a df.
This df is then saved as a csv file.

In [41]:
import sys
sys.path.insert(0,'C://Users//kuhnso//AppData//Roaming//McNeel//Rhinoceros//7.0//scripts')

In [42]:
import pickle
import pandas as pd
import os

In [43]:
#Util functions
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        try:
            os.makedirs(folder_path)
            print(f"Folder '{folder_path}' created successfully.")
        except OSError as e:
            print(f"Error creating folder '{folder_path}': {e}")

In [44]:
# def extraction function
# TOdo dont get the max but the absolut max values! or get both min and max
def extract_max_res(structure, ID):
    # initialise result dict
    max_res={}
    
    #id
    max_res['id'] =ID
    
    #um = total displacements
    max_res['um'] = max(structure.results['step_2']['nodal']['um'].values())
    #print(max_res['um'])
    
    #ux = displacements
    max_res['ux'] = max(structure.results['step_2']['nodal']['ux'].values())
    #uy = displacements
    max_res['uy'] = max(structure.results['step_2']['nodal']['uy'].values())
    #uz = displacements
    max_res['uz'] = max(structure.results['step_2']['nodal']['uz'].values())

    # sf1 = membrane foces in local x-direction nx
    max_res['sf1'] = max(structure.results['step_2']['element']['sf1'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    # sf2 = membrane forces in local y-direction ny
    max_res['sf2'] = max(structure.results['step_2']['element']['sf2'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    # sf3 = shear forces in local xy-direction nxy
    max_res['sf3'] = max(structure.results['step_2']['element']['sf3'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    # sf4 = transverse shear forces on local x-plane vy
    max_res['sf4'] = max(structure.results['step_2']['element']['sf4'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    # sf5 = transverse shear forces on local y-plane vx
    max_res['sf5'] = max(structure.results['step_2']['element']['sf5'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    # sm1 = bending moments around local y-direction m_x
    max_res['sm1'] = max(structure.results['step_2']['element']['sm1'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    # sm2 = bending moments around local x-direction m_y
    max_res['sm2'] = max(structure.results['step_2']['element']['sm2'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    # sm3 = twisting moments in x- and y-directions m_xy
    max_res['sm3'] = max(structure.results['step_2']['element']['sm3'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    
    
    # Sig_x_top = membran spannung in x richtung
    #max_res['sig_x_top'] = max(structure.results['step_2']['sig_x_top'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
    
    return max_res

In [45]:
# Eingabe
idx_s=1
start_id = 0
end_id = 0

In [46]:
y_names=['id','um','ux', 'uy', 'uz', 'sf1', 'sf2', 'sf3', 'sf4', 'sf5', 'sm1', 'sm2', 'sm3','analysis_error']

# Initialize a dictionary to store the extracted data (dict of empty lists)
y_dict = {param: [] for param in y_names}
errors=[]
noResults=[]

current_directory = os.getcwd()
folder_name='CFBData'
folder_path = os.path.join(current_directory, folder_name)

for ID in range(start_id,end_id+1):
    print(ID)
    
    # construct file path
    filepath=folder_path+'\\{}_Batch\\{}_{}_CFB\\{}_{}_structure.pkl'.format(idx_s,idx_s,ID,idx_s,ID)
    
    if os.path.exists(filepath): # if structure.pkl exists for this ID
        # get structure
        with open(filepath, "rb") as pickle_file:
            structure = pickle.load(pickle_file)

        if len(structure.results['step_2']) >0: 
            # extract max values from the analysis results of this structre 
            max_res=extract_max_res(structure, ID)

            #save max results from current structure in results dict
            for y in y_names[:-1]:
                y_dict[y].append(max_res[y])
            # error handling 
            #(TODO: better error handling would be why it failed - written in error file by ansys)
            # TODO: also should not stop analysis when analysis failes!
            y_dict['analysis_error'].append(False)
            
            
        else:
            noResults.append('{}_{}'.format(idx_s, ID))

#             for y in y_names[:-2]:
#                 y_dict[y].append(None)
            
#         y_dict['noresults'].append(True)
#         y_dict['analysis_error'].append(False)
    
    else:
        
        #structure.pkl file does not exist for this does not exist for this ID
        for y in y_names[1:-1]:
                y_dict[y].append(None)
        
        y_dict['id'].append(ID)
        y_dict['analysis_error'].append(True)
        
        errors.append('{}_{}'.format(idx_s, ID))


print(y_dict)
            
y_df=pd.DataFrame.from_dict(y_dict)
    

0
{'id': [0], 'um': [0.8934290868218064], 'ux': [0.0], 'uy': [0.15597277817725097], 'uz': [0.0], 'sf1': [13.501745224], 'sf2': [0.0], 'sf3': [15.44518947601], 'sf4': [5.60327291489], 'sf5': [41.32192993164], 'sm1': [2066.97412], 'sm2': [21542.9062], 'sm3': [1122.5813], 'analysis_error': [False]}


In [15]:
# save df to csv
path =folder_path+'\\{}_Batch\\{}_SumRes.csv'.format(idx_s,idx_s)
y_df.to_csv(path,index=True) # Save csv file

#TODO also save individual y results in the individual folders! y.csv like x.csv

In [42]:
#look at errors
noResults

[]

In [43]:
#look at errors
errors

[]

In [44]:
y_df

,id,um,ux,uy,uz,sf1,sf2,sf3,sf4,sf5,sm1,sm2,sm3,analysis_error
0,0,0.893429,0.0,0.155973,0.0,13.501745,0.0,15.445189,5.603273,41.32193,2066.97412,21542.9062,1122.5813,False


# Look at individual structure files

In [50]:
idx_s=1
ID =0# construct file path
filepath=folder_path+'\\{}_Batch\\{}_{}_CFB\\{}_{}_structure.pkl'.format(idx_s,idx_s,ID,idx_s,ID)
    
    
if os.path.exists(filepath):
    # get structure
    with open(filepath, "rb") as pickle_file:
        structure = pickle.load(pickle_file)
else:
    print('File ',filepath, 'does not exist.')
    
#     print(structure.results)
#     print(len(structure.results['step_2']))

In [51]:
structure.results

{'step_2': {'nodal': {'ux': {0: -0.023995030865230395,
    1: -0.02430648319916725,
    2: -0.02594146001493498,
    3: -0.026091971603316104,
    4: -0.025475255907150447,
    5: -0.02557588620554333,
    6: -0.02578700722593552,
    7: -0.024923980064346693,
    8: -0.017223088903372593,
    9: -0.01699923808681688,
    10: -0.016267316068033385,
    11: -0.01604458122078432,
    12: -0.011560777246103101,
    13: -0.011503338694222775,
    14: -0.011512649253991286,
    15: -0.010895417124369835,
    16: -0.012262146683992774,
    17: -0.011989625748442066,
    18: -0.01187962920127918,
    19: -0.011456602714176558,
    20: -0.00370495532605098,
    21: -0.003350082566114138,
    22: -0.0030484056804219433,
    23: -0.003027912781447455,
    24: -0.02218653412208388,
    25: -0.015382962021241625,
    26: -0.01160578493020667,
    27: -0.0216889926403077,
    28: -0.017807632230226147,
    29: -0.014408787060448354,
    30: -0.014925713721468708,
    31: -0.01339367730374287,
    3

In [51]:
structure.results['step_2']['nodal']['ux'].values()

dict_values([-0.023995030865230384, -0.02430648319916725, -0.025941460014934983, -0.02609197160331611, -0.025475255907150447, -0.02557588620554333, -0.02578700722593552, -0.024923980064346693, -0.017223088903372593, -0.01699923808681688, -0.01626731606803339, -0.01604458122078432, -0.011560777246103101, -0.011503338694222775, -0.011512649253991284, -0.010895417124369835, -0.012262146683992774, -0.011989625748442064, -0.01187962920127918, -0.011456602714176558, -0.003704955326050973, -0.0033500825661141372, -0.0030484056804219433, -0.0030279127814474543, -0.022186534122083882, -0.015382962021241627, -0.011605784930206673, -0.0216889926403077, -0.01780763223022615, -0.014408787060448356, -0.014925713721468708, -0.01339367730374287, -0.012178481703350182, -0.009610429734539246, -0.0088804088588015, -0.008550926544199752, -0.009962147291822616, -0.008717166109055884, -0.007581156517705856, -0.0031042933758681016, -0.0034038232455621854, -0.003460265220928774, -0.010044183103058891, -0.0097

In [54]:
max(structure.results['step_2']['GP']['sig_x_top'].values())

8.381880967078814e-09

In [55]:
structure.results['step_2']['GP']['eps_1_bot']

{0: -5.686425907924786e-08,
 1: -1.8267639845248596e-08,
 2: 3.3254875120292127e-07,
 3: 3.744032994745794e-07,
 4: -9.669806644296841e-08,
 5: 1.044272295303017e-07,
 6: 1.9011918084288482e-06,
 7: 2.1185878722462803e-06,
 8: 1.3857215890311636e-06,
 9: 1.6932054904827964e-06,
 10: 2.8887159260193584e-06,
 11: 3.2176201330003096e-06,
 12: 2.6980098482454196e-05,
 13: 2.5891999030136503e-05,
 14: 2.661176404217258e-05,
 15: 2.5503528377157636e-05,
 16: 2.597765706013888e-05,
 17: 2.4800545361358672e-05,
 18: 2.5311263016192243e-05,
 19: 2.4110147933242843e-05,
 20: 2.4182490960811265e-05,
 21: 2.283705907757394e-05,
 22: 2.301507083757315e-05,
 23: 2.161502379749436e-05,
 24: 1.761923158483114e-05,
 25: 1.730965232127346e-05,
 26: 1.7763752111932263e-05,
 27: 1.7452317479182966e-05,
 28: 1.8691916920943186e-05,
 29: 1.959410474228207e-05,
 30: 1.8174405340687372e-05,
 31: 1.906298530229833e-05,
 32: 2.0607820260920562e-05,
 33: 2.1036921680206433e-05,
 34: 2.0438010324141942e-05,
 35: 

In [53]:
abs(min(structure.results['step_2']['GP']['sig_x_top'].values()))

0.05364295840263367

In [54]:
structure.results['step_2']['GP']['fcc_eff_bot']

{0: 13.103707313537598,
 1: 13.103707313537598,
 2: 13.10338020324707,
 3: 13.103339195251465,
 4: 13.103707313537598,
 5: 13.103604316711426,
 6: 13.101839065551758,
 7: 13.101625442504883,
 8: 13.10234546661377,
 9: 13.102043151855469,
 10: 13.100868225097656,
 11: 13.100545883178711,
 12: 13.077244758605957,
 13: 13.078310012817383,
 14: 13.077605247497559,
 15: 13.078690528869629,
 16: 13.078226089477539,
 17: 13.079379081726074,
 18: 0.0,
 19: 13.080055236816406,
 20: 13.079983711242676,
 21: 13.08130168914795,
 22: 13.081127166748047,
 23: 13.082498550415039,
 24: 13.086414337158203,
 25: 13.08671760559082,
 26: 13.086272239685059,
 27: 13.086577415466309,
 28: 13.085362434387207,
 29: 13.084478378295898,
 30: 13.085869789123535,
 31: 13.084999084472656,
 32: 13.08348560333252,
 33: 13.083065032958984,
 34: 13.083651542663574,
 35: 13.083229064941406,
 36: 13.1032133102417,
 37: 13.10318374633789,
 38: 13.103422164916992,
 39: 13.10339641571045,
 40: 13.103053092956543,
 41: 13.1

#### Pseudocode

###### Biegenachweis (nur beton spannungen)
for structure in structures:

    for i in allen shell emenets: (not MPCs!)
    
        # NW top
        # get max of sig_x_top, sig_y_top (beides druck)
        # auch zug spannung überprüfen?
        # get fcc,eff_top
        #calculate ausnutzung
        fcc_eff_top / sig_x_top
        
        # NW top
        # get sig_x_bot, sig_y_bot
        
        # safe max (NW top und bottom)
        
    extract maximal max aisnutzung aller elemente der structure

###### Momenten nachweis
for i in all schell elements: (not MPCs!)

    #get bending moments in x and y direction (Med)
    structure.results[step2][elements][sm1/2] (--> sign decides which resistance has to be calculated)
   
    # Read out necessary attributes for element i
    propertie=structure.element[i].property
    section = structure.properties[i].section
    t=structure.section[section]
    section = structure.properties[i].material
    d1=structure.material[material].d1
    as =...
    ....
    
    #calculate Resistance Mrd (in negative an postive moment direction, and in x and y direction --> 4 resistances)
    d = t-cnom-d/2 (-d1) (unterschied innere oder äussere lage)
    Mrd = as *fsd *(d-(as*fsd/2*b*fcd)) (b für eine Platte =1m setze?)
    
    
    # calculate Ausnutung
    mu_mx=Mrd/Med (--> should be larger 1
    
    


In [18]:
print(structure.elements)

{0: ShellElement(0), 1: ShellElement(1), 2: ShellElement(2), 3: ShellElement(3), 4: ShellElement(4), 5: ShellElement(5), 6: ShellElement(6), 7: ShellElement(7), 8: ShellElement(8), 9: ShellElement(9), 10: ShellElement(10), 11: ShellElement(11), 12: ShellElement(12), 13: ShellElement(13), 14: ShellElement(14), 15: ShellElement(15), 16: ShellElement(16), 17: ShellElement(17), 18: ShellElement(18), 19: ShellElement(19), 20: ShellElement(20), 21: ShellElement(21), 22: ShellElement(22), 23: ShellElement(23), 24: ShellElement(24), 25: ShellElement(25), 26: ShellElement(26), 27: ShellElement(27), 28: ShellElement(28), 29: ShellElement(29), 30: ShellElement(30), 31: ShellElement(31), 32: ShellElement(32), 33: ShellElement(33), 34: ShellElement(34), 35: ShellElement(35), 36: ShellElement(36), 37: ShellElement(37), 38: ShellElement(38), 39: ShellElement(39), 40: ShellElement(40), 41: ShellElement(41), 42: ShellElement(42), 43: ShellElement(43), 44: ShellElement(44), 45: ShellElement(45), 46: She

In [19]:
structure.loads

{'earthPressure_liveLoad': AreaLoad(earthPressure_liveLoad),
 'Gleis1_EIGENGEWICHTE_SCHIENE_Lasteinzugsflache': AreaLoad(Gleis1_EIGENGEWICHTE_SCHIENE_Lasteinzugsflache),
 'earthPressure_backfill': AreaLoad(earthPressure_backfill),
 'Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_2': AreaLoad(Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_2),
 'Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_-1': AreaLoad(Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_-1),
 'Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_1': AreaLoad(Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_1),
 'load_gravity': GravityLoad(load_gravity)}

In [19]:
structure.nodes

{0: Node(0),
 1: Node(1),
 2: Node(2),
 3: Node(3),
 4: Node(4),
 5: Node(5),
 6: Node(6),
 7: Node(7),
 8: Node(8),
 9: Node(9),
 10: Node(10),
 11: Node(11),
 12: Node(12),
 13: Node(13),
 14: Node(14),
 15: Node(15),
 16: Node(16),
 17: Node(17),
 18: Node(18),
 19: Node(19),
 20: Node(20),
 21: Node(21),
 22: Node(22),
 23: Node(23),
 24: Node(24),
 25: Node(25),
 26: Node(26),
 27: Node(27),
 28: Node(28),
 29: Node(29),
 30: Node(30),
 31: Node(31),
 32: Node(32),
 33: Node(33),
 34: Node(34),
 35: Node(35),
 36: Node(36),
 37: Node(37),
 38: Node(38),
 39: Node(39),
 40: Node(40),
 41: Node(41),
 42: Node(42),
 43: Node(43),
 44: Node(44),
 45: Node(45),
 46: Node(46),
 47: Node(47),
 48: Node(48),
 49: Node(49),
 50: Node(50),
 51: Node(51),
 52: Node(52),
 53: Node(53),
 54: Node(54),
 55: Node(55),
 56: Node(56),
 57: Node(57),
 58: Node(58),
 59: Node(59),
 60: Node(60),
 61: Node(61),
 62: Node(62),
 63: Node(63),
 64: Node(64),
 65: Node(65),
 66: Node(66),
 67: Node(67),


In [20]:
# Import structure
filepath='C:\\Temp\\02_AnalysisResults\\1_19_structure.pkl'

# Load the data from the pickle file
with open(filepath, "rb") as pickle_file:
    structure = pickle.load(pickle_file)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Temp\\02_AnalysisResults\\1_19_structure.pkl'

In [ ]:
max(structure.results['step_2']['element']['sf2'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']

In [ ]:
structure.results['step_2']['element']['sf2']

In [ ]:
# get maximum Values form analysis results saved in the structure object

#um = total displacements
um=max(structure.results['step_2']['nodal']['um'].values())

# sf1 = membrane foces in local x-direction nx
Sf1= max(structure.results['step_2']['element']['sf1'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
# sf2 = membrane forces in local y-direction ny
Sf2= max(structure.results['step_2']['element']['sf2'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
# sf3 = shear forces in local xy-direction nxy
Sf3= max(structure.results['step_2']['element']['sf3'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
# sf4 = transverse shear forces on local x-plane vy
Sf4= max(structure.results['step_2']['element']['sf4'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
# sf5 = transverse shear forces on local y-plane vx
Sf5= max(structure.results['step_2']['element']['sf5'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
# sm1 = bending moments around local y-direction m_x
sm1= max(structure.results['step_2']['element']['sm1'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
# sm2 = bending moments around local x-direction m_y
sm2= max(structure.results['step_2']['element']['sm2'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']
# sm3 = twisting moments in x- and y-directions m_xy
sm3= max(structure.results['step_2']['element']['sm3'].values(), key=lambda x: x['ip1_sp0'])['ip1_sp0']

Ziel: data = {'col_1': [3, 2, 1, 0], 'col_2': ['a', 'b', 'c', 'd']}
    --> put all sf1 in a dict, etc.dd

# Structure

In [24]:
structure_dict = vars(structure)

In [25]:
structure_dict

{'constraints': {},
 'displacements': {'nset_pinned_set_disp_1': GeneralDisplacement(nset_pinned_set_disp_1),
  'nset_pinned_set_disp_2': GeneralDisplacement(nset_pinned_set_disp_2)},
 'elements': {0: ShellElement(0),
  1: ShellElement(1),
  2: ShellElement(2),
  3: ShellElement(3),
  4: ShellElement(4),
  5: ShellElement(5),
  6: ShellElement(6),
  7: ShellElement(7),
  8: ShellElement(8),
  9: ShellElement(9),
  10: ShellElement(10),
  11: ShellElement(11),
  12: ShellElement(12),
  13: ShellElement(13),
  14: ShellElement(14),
  15: ShellElement(15),
  16: ShellElement(16),
  17: ShellElement(17),
  18: ShellElement(18),
  19: ShellElement(19),
  20: ShellElement(20),
  21: ShellElement(21),
  22: ShellElement(22),
  23: ShellElement(23),
  24: ShellElement(24),
  25: ShellElement(25),
  26: ShellElement(26),
  27: ShellElement(27),
  28: ShellElement(28),
  29: ShellElement(29),
  30: ShellElement(30),
  31: ShellElement(31),
  32: ShellElement(32),
  33: ShellElement(33),
  34: Sh

In [32]:
structure_dict['results']

{'step_2': {'nodal': {'ux': {0: -0.023995030865230395,
    1: -0.02430648319916725,
    2: -0.02594146001493498,
    3: -0.026091971603316104,
    4: -0.025475255907150447,
    5: -0.02557588620554333,
    6: -0.02578700722593552,
    7: -0.024923980064346693,
    8: -0.017223088903372593,
    9: -0.01699923808681688,
    10: -0.016267316068033385,
    11: -0.01604458122078432,
    12: -0.011560777246103101,
    13: -0.011503338694222775,
    14: -0.011512649253991286,
    15: -0.010895417124369835,
    16: -0.012262146683992774,
    17: -0.011989625748442066,
    18: -0.01187962920127918,
    19: -0.011456602714176558,
    20: -0.00370495532605098,
    21: -0.003350082566114138,
    22: -0.0030484056804219433,
    23: -0.003027912781447455,
    24: -0.02218653412208388,
    25: -0.015382962021241625,
    26: -0.01160578493020667,
    27: -0.0216889926403077,
    28: -0.017807632230226147,
    29: -0.014408787060448354,
    30: -0.014925713721468708,
    31: -0.01339367730374287,
    3

In [40]:
structure.results['step_2']['GP'].keys()

dict_keys(['elem_nr_top', 'sig_y_bot', 'elem_typ', 'loc_y_glob_y', 'GP_name_top', 'loc_x_glob_x', 'coor_intp_layer_x_bot', 'tau_xy_top', 'coor_intp_layer_y_bot', 'nr', 'loc_x_glob_z', 'fcc_eff_bot', 'sig_x_top', 'coor_intp_layer_z_bot', 'tau_xy_bot', 'loc_y_glob_x', 'sig_y_top', 'GP_name_bot', 'elem_nr_bot', 'loc_y_glob_z', 'loc_x_glob_y', 'coor_intp_layer_x_top', 'fcc_eff_top', 'coor_intp_layer_y_top', 'coor_intp_layer_z_top', 'sig_x_bot'])

# Nodes

In [57]:
print(structure.nodes)

{0: Node(0), 1: Node(1), 2: Node(2), 3: Node(3), 4: Node(4), 5: Node(5), 6: Node(6), 7: Node(7), 8: Node(8), 9: Node(9), 10: Node(10), 11: Node(11), 12: Node(12), 13: Node(13), 14: Node(14), 15: Node(15), 16: Node(16), 17: Node(17), 18: Node(18), 19: Node(19), 20: Node(20), 21: Node(21), 22: Node(22), 23: Node(23), 24: Node(24), 25: Node(25), 26: Node(26), 27: Node(27), 28: Node(28), 29: Node(29), 30: Node(30), 31: Node(31), 32: Node(32), 33: Node(33), 34: Node(34), 35: Node(35), 36: Node(36), 37: Node(37), 38: Node(38), 39: Node(39), 40: Node(40), 41: Node(41), 42: Node(42), 43: Node(43), 44: Node(44), 45: Node(45), 46: Node(46), 47: Node(47), 48: Node(48), 49: Node(49), 50: Node(50), 51: Node(51), 52: Node(52), 53: Node(53), 54: Node(54), 55: Node(55), 56: Node(56), 57: Node(57), 58: Node(58), 59: Node(59), 60: Node(60), 61: Node(61), 62: Node(62), 63: Node(63), 64: Node(64), 65: Node(65), 66: Node(66), 67: Node(67), 68: Node(68), 69: Node(69), 70: Node(70), 71: Node(71), 72: Node(72

In [58]:
print(structure.nodes[0])



compas_fea Node object
----------------------
key   : 0
x     : 0.0
y     : 0.0
z     : 0.0
ex    : [1, 0, 0]
ey    : [0, 1, 0]
ez    : [0, 0, 1]
mass  : None



In [59]:
print(vars(structure.nodes[10]))

{'__name__': 'Node', 'key': 10, 'x': 2138.4, 'y': 1782.0, 'z': 0.0, 'ex': [1, 0, 0], 'ey': [0, 1, 0], 'ez': [0, 0, 1], 'mass': None}


# Elements

In [60]:
print(structure.elements)

{0: ShellElement(0), 1: ShellElement(1), 2: ShellElement(2), 3: ShellElement(3), 4: ShellElement(4), 5: ShellElement(5), 6: ShellElement(6), 7: ShellElement(7), 8: ShellElement(8), 9: ShellElement(9), 10: ShellElement(10), 11: ShellElement(11), 12: ShellElement(12), 13: ShellElement(13), 14: ShellElement(14), 15: ShellElement(15), 16: ShellElement(16), 17: ShellElement(17), 18: ShellElement(18), 19: ShellElement(19), 20: ShellElement(20), 21: ShellElement(21), 22: ShellElement(22), 23: ShellElement(23), 24: ShellElement(24), 25: ShellElement(25), 26: ShellElement(26), 27: ShellElement(27), 28: ShellElement(28), 29: ShellElement(29), 30: ShellElement(30), 31: ShellElement(31), 32: ShellElement(32), 33: ShellElement(33), 34: ShellElement(34), 35: ShellElement(35), 36: ShellElement(36), 37: ShellElement(37), 38: ShellElement(38), 39: ShellElement(39), 40: ShellElement(40), 41: ShellElement(41), 42: ShellElement(42), 43: ShellElement(43), 44: ShellElement(44), 45: ShellElement(45), 46: She

In [61]:
print(structure.elements[6])



compas_fea ShellElement object
------------------------------
nodes      : [12, 13, 17, 16]
number     : 6
thermal    : False
axes       : {'ey': None, 'ez': None, 'ex': None}
element_property : elset_plate_2_element_prop



In [62]:
print(structure.elements[6].element_property)

elset_plate_2_element_prop


# Element Properties

In [63]:
structure.element_properties

{'elset_wall_2_4_element_prop': ElementProperties(elset_wall_2_4_element_prop),
 'elset_wall_1_2_element_prop': ElementProperties(elset_wall_1_2_element_prop),
 'elset_wall_1_8_element_prop': ElementProperties(elset_wall_1_8_element_prop),
 'elset_wall_2_2_element_prop': ElementProperties(elset_wall_2_2_element_prop),
 'elset_plate_1_element_prop': ElementProperties(elset_plate_1_element_prop),
 'elset_wall_2_5_element_prop': ElementProperties(elset_wall_2_5_element_prop),
 'elset_plate_9_element_prop': ElementProperties(elset_plate_9_element_prop),
 'elset_wall_1_6_element_prop': ElementProperties(elset_wall_1_6_element_prop),
 'elset_plate_5_element_prop': ElementProperties(elset_plate_5_element_prop),
 'elset_wall_1_3_element_prop': ElementProperties(elset_wall_1_3_element_prop),
 'elset_plate_3_element_prop': ElementProperties(elset_plate_3_element_prop),
 'elset_wall_2_3_element_prop': ElementProperties(elset_wall_2_3_element_prop),
 'elset_wall_2_6_element_prop': ElementPropertie

In [23]:
vars(structure.element_properties['elset_wall_2_4_element_prop'])

{'index': 21,
 '__name__': 'ElementProperties',
 'name': 'elset_wall_2_4_element_prop',
 'material': 'wall_element_mat',
 'section': 'wall2_element_sec',
 'elset': 'elset_wall_2_4',
 'elements': None,
 'rebar': {}}

# Sections

In [22]:
structure.sections

{'wall1_element_sec': ShellSection(wall1_element_sec),
 'plate_element_sec': ShellSection(plate_element_sec),
 'wall2_element_sec': ShellSection(wall2_element_sec),
 'sec_mpc': MPCSection(sec_mpc)}

In [21]:
print(structure.sections['elset_wall_2_4_element_sec'])

KeyError: 'elset_wall_2_4_element_sec'

In [67]:
vars(structure.sections['elset_wall_2_4_element_sec'])

KeyError: 'elset_wall_2_4_element_sec'

# Materials

In [68]:
structure.materials


{'elset_mpc_element_mat': MPCStiff(elset_mpc_element_mat),
 'wall_element_mat': CMMUsermat(wall_element_mat),
 'plate_element_mat': CMMUsermat(plate_element_mat)}

In [69]:
print(structure.materials['elset_wall_1_4_element_mat'].attr_list)

KeyError: 'elset_wall_1_4_element_mat'

In [70]:
print(structure.materials['elset_wall_1_4_element_mat'])

KeyError: 'elset_wall_1_4_element_mat'

In [71]:
print(vars(structure.materials['elset_wall_1_4_element_mat']))

KeyError: 'elset_wall_1_4_element_mat'

# Loads

In [72]:
structure.loads

{'earthPressure_liveLoad': AreaLoad(earthPressure_liveLoad),
 'Gleis1_EIGENGEWICHTE_SCHIENE_Lasteinzugsflache': AreaLoad(Gleis1_EIGENGEWICHTE_SCHIENE_Lasteinzugsflache),
 'earthPressure_backfill': AreaLoad(earthPressure_backfill),
 'Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_2': AreaLoad(Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_2),
 'Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_-1': AreaLoad(Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_-1),
 'Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_1': AreaLoad(Gleis1_BAHNLASTEN_Lasteinzugsflache_Lastblock_1),
 'load_gravity': GravityLoad(load_gravity)}

# Results

In [73]:
structure.results

{'step_2': {'nodal': {'ux': {0: -0.023995030865230384,
    1: -0.02430648319916725,
    2: -0.025941460014934983,
    3: -0.02609197160331611,
    4: -0.025475255907150447,
    5: -0.02557588620554333,
    6: -0.02578700722593552,
    7: -0.024923980064346693,
    8: -0.017223088903372593,
    9: -0.01699923808681688,
    10: -0.01626731606803339,
    11: -0.01604458122078432,
    12: -0.011560777246103101,
    13: -0.011503338694222775,
    14: -0.011512649253991284,
    15: -0.010895417124369835,
    16: -0.012262146683992774,
    17: -0.011989625748442064,
    18: -0.01187962920127918,
    19: -0.011456602714176558,
    20: -0.003704955326050973,
    21: -0.0033500825661141372,
    22: -0.0030484056804219433,
    23: -0.0030279127814474543,
    24: -0.022186534122083882,
    25: -0.015382962021241627,
    26: -0.011605784930206673,
    27: -0.0216889926403077,
    28: -0.01780763223022615,
    29: -0.014408787060448356,
    30: -0.014925713721468708,
    31: -0.01339367730374287,
  

In [74]:
import math as m
print(m.sqrt(0.008979894650936805**2 + 0.011555043702011514**2 +0.0768866718939348**2))

0.0782669653035232


In [75]:
# nodal results
# include displacement of the nodes (in x,y and z direction)
# um is the total displacement (m.sqrt(ux**2 + uy**2 +uz**2))
# a displacement of 0 is the case for fixed supports for example with no initial displacement)
structure.results['step_2']['nodal']['um']

{0: 0.09911600027401393,
 1: 0.08893613357237246,
 2: 0.07726880447173748,
 3: 0.05844909493043591,
 4: 0.40957837188935786,
 5: 0.37499662078901,
 6: 0.3314302985777835,
 7: 0.2749342631022718,
 8: 0.7633604187863602,
 9: 0.7012895070610159,
 10: 0.6236290377986561,
 11: 0.5267815890576409,
 12: 0.7025352035016441,
 13: 0.648842791988031,
 14: 0.5863147909255686,
 15: 0.5115842151096872,
 16: 0.33453313065907586,
 17: 0.3105489369781667,
 18: 0.2845129348766294,
 19: 0.2526829841513062,
 20: 0.04231329444387888,
 21: 0.03748082117437577,
 22: 0.03359759727756594,
 23: 0.029883265678422054,
 24: 0.03870461849578978,
 25: 0.026814527959055114,
 26: 0.02284313247028341,
 27: 0.21878403570888447,
 28: 0.1821834000392651,
 29: 0.16601852746106774,
 30: 0.4323655857646075,
 31: 0.36685162213509687,
 32: 0.33402809918808946,
 33: 0.4329310118609973,
 34: 0.3703007909234811,
 35: 0.3354325497503232,
 36: 0.21601544165314318,
 37: 0.18465328550068572,
 38: 0.16715474572906255,
 39: 0.025817554

In [76]:
# element results
# elements include shell elements and MPCs
# some elements are shell elements and some are MPCs (the MPCs don't have any membrane forces etc.)
# sf1 = membrane foces in local x-direction nx
# sf2 = membrane forces in local y-direction ny
# sf3 = shear forces in local xy-direction nxy
# sf4 = transverse shear forces on local x-plane vy
# sf5 = transverse shear forces on local y-plane vx
# sm1 = bending moments around local y-direction m_x
# sm2 = bending moments around local x-direction m_y
# sm3 = twisting moments in x- and y-directions m_xy

# Was ist ip1_sp0 - integrationspunkt 1?
structure.results['step_2']['element']['sm2']

{0: {'ip1_sp0': 18611.9824},
 1: {'ip1_sp0': 17155.6934},
 2: {'ip1_sp0': 15293.6387},
 3: {'ip1_sp0': -11906.4492},
 4: {'ip1_sp0': -10831.4609},
 5: {'ip1_sp0': -9264.7373},
 6: {'ip1_sp0': -8068.55176},
 7: {'ip1_sp0': -7807.80322},
 8: {'ip1_sp0': -7830.70557},
 9: {'ip1_sp0': 19350.2949},
 10: {'ip1_sp0': 17740.2266},
 11: {'ip1_sp0': 15842.0723},
 12: {'ip1_sp0': 13488.2559},
 13: {'ip1_sp0': 11759.6777},
 14: {'ip1_sp0': 10596.1709},
 15: {'ip1_sp0': -7638.88477},
 16: {'ip1_sp0': -6221.03809},
 17: {'ip1_sp0': -5557.06689},
 18: {'ip1_sp0': -7553.33789},
 19: {'ip1_sp0': -6706.19678},
 20: {'ip1_sp0': -5868.19141},
 21: {'ip1_sp0': 13749.8838},
 22: {'ip1_sp0': 11833.6523},
 23: {'ip1_sp0': 10589.165},
 24: {'ip1_sp0': 10048.1904},
 25: {'ip1_sp0': 9887.82715},
 26: {'ip1_sp0': 9917.53516},
 27: {'ip1_sp0': -5431.89404},
 28: {'ip1_sp0': -5363.14795},
 29: {'ip1_sp0': -5324.99805},
 30: {'ip1_sp0': -5522.4292},
 31: {'ip1_sp0': -5479.99512},
 32: {'ip1_sp0': -5473.29688},
 33: 

In [77]:
# read out results see: C:\Users\kuhnso\AppData\Roaming\McNeel\Rhinoceros\7.0\scripts\compas_fea\fea\ansys_sel\ansys_sel.py

# GP= Gaus Punkte (Integration Punkte)
# Shell elements haben je 4 GP
# MPCs haben auch je 4 GP
# also gibt es element number * 4 GP insgesamt

# elem_typ: definiert element typ für jeden GP (shell element =1, MPC element =0)
# nr:  nummerierung (startet bei 1) -meaning???

# GP_name_top: definiert den name (die nummer) des GP "top" (note: fängt von 1 an)
# elem_nr_top: definiert die zugehörigkeit vom GP zum Element (! hier fangen die Element nummern bei 1 an und nicht bei 0)
#                                                                  und bei MPC elements steht nicht die element nummer sondern o?
# sig_x_top: membran spannung in x richtung (am "top")
# sig_y_top: membran spannung in y richtung (am "top")
# tau_xy_top: shear spannung in der plattenebene (am "top")  was ist mit tau_zy/zx????
# fcc_eff_top: fcc= compressive strength of concrete, effective  ??

# coor_intp_layer_x_top ???
# coor_intp_layer_y_top
# coor_intp_layer_z_top

# GP_name_bot
# elem_nr_bot:  definiert die zugehörigkeit vom GP zum Element (! hier fangen die Element nummern bei 1 an und nicht bei 0)
#                                                                  und bei MPC elements steht nicht die element nummer sondern o?
# sig_y_bot
# sig_x_bot
# tau_xy_bot
# fcc_eff_bot

# coor_intp_layer_x_bot
# coor_intp_layer_y_bot
# coor_intp_layer_z_bot

# Principal Stresses at each GP
# loc_y_glob_y  ???
# loc_y_glob_x
# loc_y_glob_z
# loc_x_glob_y
# loc_x_glob_x
# loc_x_glob_z



structure.results['step_2']['GP']['sig_x_top']

{0: -0.013888312503695488,
 1: -0.014618699438869953,
 2: -0.019669918343424797,
 3: -0.020702233538031578,
 4: -0.03916454687714577,
 5: -0.0417538546025753,
 6: -0.050203219056129456,
 7: -0.05364295840263367,
 8: -0.027173426002264023,
 9: -0.02952047437429428,
 10: -0.03926808014512062,
 11: -0.04261064529418945,
 12: -0.008257482200860977,
 13: -0.008017502725124359,
 14: -0.011634071357548237,
 15: -0.011394557543098927,
 16: -0.036196477711200714,
 17: -0.03569973260164261,
 18: -0.039916470646858215,
 19: -0.03942092880606651,
 20: -0.015152906067669392,
 21: -0.01565852016210556,
 22: -0.02087395265698433,
 23: -0.021379470825195312,
 24: -0.008350895717740059,
 25: -0.007541733328253031,
 26: -0.006565754767507315,
 27: -0.007320060860365629,
 28: -0.022266415879130363,
 29: -0.01951805129647255,
 30: -0.011133119463920593,
 31: -0.013154694810509682,
 32: -0.017427857965230942,
 33: -0.017998598515987396,
 34: -0.017561834305524826,
 35: -0.018132459372282028,
 36: -0.010506

In [19]:
# later zum stahl spannungen raus lesen
structure.results['step_2']['GP']['sig_sr_layer_4']

KeyError: 'sig_sr_layer_4'

In [20]:
structure.results['step_2']['GP'].keys()

dict_keys(['elem_nr_top', 'sig_y_bot', 'elem_typ', 'loc_y_glob_y', 'GP_name_top', 'loc_x_glob_x', 'coor_intp_layer_x_bot', 'tau_xy_top', 'coor_intp_layer_y_bot', 'nr', 'loc_x_glob_z', 'fcc_eff_bot', 'sig_x_top', 'coor_intp_layer_z_bot', 'tau_xy_bot', 'loc_y_glob_x', 'sig_y_top', 'GP_name_bot', 'elem_nr_bot', 'loc_y_glob_z', 'loc_x_glob_y', 'coor_intp_layer_x_top', 'fcc_eff_top', 'coor_intp_layer_y_top', 'coor_intp_layer_z_top', 'sig_x_bot'])